In [ ]:
import kagglehub

# Download latest version
path = kagglehub.model_download("mistral-ai/mistral/pyTorch/7b-instruct-v0.1-hf")

print("Path to model files:", path)


  0%|          | 0.00/4.72G [00:00<?, ?B/s]

Download already complete (1795303 bytes).
Download already complete (72 bytes).
Download already complete (1519 bytes).
Download already complete (116 bytes).
Download already complete (23950 bytes).




  0%|          | 0.00/9.26G [00:00<?, ?B/s]

Download already complete (571 bytes).



  0%|          | 1.00M/4.72G [00:01<1:47:53, 782kB/s]

  0%|          | 1.00M/9.26G [00:01<3:36:12, 766kB/s]
  0%|          | 2.00M/4.72G [00:01<55:11, 1.53MB/s] 
  0%|          | 3.00M/4.72G [00:01<34:58, 2.41MB/s]

  0%|          | 2.00M/9.26G [00:01<1:50:32, 1.50MB/s]
  0%|          | 5.00M/4.72G [00:01<19:00, 4.43MB/s]

  0%|          | 3.00M/9.26G [00:01<1:09:45, 2.38MB/s]

Download already complete (1466 bytes).



  0%|          | 7.00M/4.72G [00:01<12:28, 6.75MB/s]

  0%|          | 5.00M/9.26G [00:01<37:48, 4.38MB/s]  
  0%|          | 9.00M/4.72G [00:02<09:47, 8.60MB/s]

Download already complete (493443 bytes).


Streaming output truncated to the last 5000 lines.
 59%|█████▊    | 2.76G/4.72G [03:04<02:10, 16.1MB/s]

 30%|███       | 2.78G/9.26G [03:04<06:56, 16.7MB/s]
 59%|█████▊    | 2.76G/4.72G [03:04<02:08, 16.3MB/s]
 59%|█████▊    | 2.76G/4.72G [03:04<02:06, 16.6MB/s]

 30%|███       | 2.79G/9.26G [03:04<06:50, 16.9MB/s]
 59%|█████▊    | 2.77G/4.72G [03:04<02:05, 16.8MB/s]
 59%|█████▊    | 2.77G/4.72G [03:04<02:12, 15.8MB/s]

 30%|███       | 2.79G/9.26G [03:04<07:04, 16.4MB/s]
 59%|█████▊    | 2.77G/4.72G [03:05<02:22, 14.7MB/s]

 30%|███       | 2.79G/9.26G [03:05<06:53, 16.8MB/s]
 59%|█████▉    | 2.77G/4.72G [03:05<02:04, 16.7MB/s]
 59%|█████▉    | 2.77G/4.72G [03:05<02:03, 16.9MB/s]

 30%|███       | 2.80G/9.26G [03:05<06:48, 17.0MB/s]
 59%|█████▉    | 2.78G/4.72G [03:05<02:02, 17.0MB/s]
 59%|█████▉    | 2.78G/4.72G [03:05<02:10, 15.9MB/s]

 30%|███       | 2.80G/9.26G [03:05<07:02, 16.4MB/s]
 59%|█████▉    | 2.78G/4.72G [03:05<02:21, 14.7MB/s]

 30%|███       | 2.80G/9.26G [03:05<06:52

Path to model files: /root/.cache/kagglehub/models/mistral-ai/mistral/pyTorch/7b-instruct-v0.1-hf/1


In [1]:
%%capture
!pip install -U bitsandbytes
!pip install -U transformers
!pip install -U peft
!pip install -U accelerate
!pip install -U trl

  Using cached peft-0.11.1-py3-none-any.whl (251 kB)


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig,HfArgumentParser,TrainingArguments,pipeline, logging
from peft import LoraConfig, PeftModel, prepare_model_for_kbit_training, get_peft_model
import os,torch, wandb
from datasets import load_dataset
from trl import SFTTrainer

In [ ]:
from kaggle_secrets import UserSecretsClient
user_secrets = UserSecretsClient()
secret_hf = user_secrets.get_secret("HF_TOKEN")
secret_wandb = user_secrets.get_secret("wandb")

In [ ]:
!huggingface-cli login --token $secret_hf

In [ ]:
# Monitering the LLM
wandb.login(key = secret_wandb)
run = wandb.init(
    project='Fine-Tuning Mistral Instruct 7B With Q Lora',
    job_type="training",
    anonymous="allow"
)

In [ ]:
base_model = "/kaggle/input/mistral/pytorch/7b-instruct-v0.1-hf/1"
dataset_name = "mlabonne/guanaco-llama2-1k"
new_model = "mistral_7b-instruct-guanaco"

In [ ]:
#Importing the dataset
dataset = load_dataset(dataset_name, split="train")
dataset["text"][100]

In [ ]:
# Load base model(Mistral 7B)
bnb_config = BitsAndBytesConfig(
    load_in_4bit= True,
    bnb_4bit_quant_type= "nf4",
    bnb_4bit_compute_dtype= torch.bfloat16,
    bnb_4bit_use_double_quant= False,
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
        base_model,
        quantization_config=bnb_config,
        torch_dtype=torch.bfloat16,
        device_map="auto",
        trust_remote_code=True,
)

In [ ]:
model.config.use_cache = False # silence the warnings. Please re-enable for inference!
model.config.pretraining_tp = 1
model.gradient_checkpointing_enable()

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)
tokenizer.padding_side = 'right'
tokenizer.pad_token = tokenizer.eos_token
tokenizer.add_eos_token = True
tokenizer.add_bos_token, tokenizer.add_eos_token

In [ ]:
#Adding the adapters in the layers
model = prepare_model_for_kbit_training(model)
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj","gate_proj"]
)
model = get_peft_model(model, peft_config)

In [ ]:
#Hyperparamter
training_arguments = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="wandb"
)

In [ ]:
# Setting sft parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length= None,
    dataset_text_field="text",
    tokenizer=tokenizer,
    args=training_arguments,
    packing= False,
)

In [ ]:
trainer.train()

In [ ]:
# Save the fine-tuned model
trainer.model.save_pretrained(new_model)
wandb.finish()
model.config.use_cache = True
model.eval()

In [ ]:
trainer.model.push_to_hub("rizwan-ai/mistral_7b-instruct-guanaco", use_temp_dir=False)

In [ ]:
logging.set_verbosity(logging.CRITICAL)

In [ ]:
prompt = "How do I find true love?"
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"<s>[INST] {prompt} [/INST]")
print(result[0]['generated_text'])